In [ ]:
def read_undet_indices(result_path):
    undet_indices = []
    with open(result_path,'r') as f:
        header_found = False
        for line in f:
            # Strip leading/trailing whitespace characters
            line = line.strip()
            
            # Check if the current line is the header
            if line == "# Undetected object indices":
                header_found = True
                continue
            
            # If the header is found, store the subsequent lines
            if header_found:
                undet_indices.append(line)
    return undet_indices

cgc_result_path = '/mnt/2tb-1/satellitepy/fineair_experiments/roi_trans_cgc/evaluations/mAP_values.txt'
fac_result_path = '/mnt/2tb-1/satellitepy/fineair_experiments/roi_trans_fac/evaluations/mAP_values.txt'
cgc_undet_indices = read_undet_indices(cgc_result_path)
fac_undet_indices = read_undet_indices(fac_result_path)
# Print the stored lines (optional)
# Find overlapping items (intersection)
overlapping_items = list(set(cgc_undet_indices) & set(fac_undet_indices))

# Find non-overlapping items in List 1
nonoverlapping_list1 = list(set(cgc_undet_indices) - set(fac_undet_indices))

# Find non-overlapping items in List 2
nonoverlapping_list2 = list(set(fac_undet_indices) - set(cgc_undet_indices))

# Print the results
print("Total # Undetected objects by the CGC model:", len(cgc_undet_indices))
print("Total # Undetected objects by the FAC model:", len(fac_undet_indices))
print("# Undetected objects by both:", len(overlapping_items))
print("# Undetected objects only by the CGC model:", len(nonoverlapping_list1))
print("# Undetected objects only by the FAC model:", len(nonoverlapping_list2))



# fix tobias' result files

In [1]:
import json
from pathlib import Path
import numpy as np
from satellitepy.data.utils import get_satellitepy_table


In [4]:
in_folders = ['/home/murat/Projects/satellitepy/exps/tobias_SR/v1/Fainair_Lables_dota_enhx4',
'/home/murat/Projects/satellitepy/exps/tobias_SR/v1/Fainair_Lables_dota_enh',
'/home/murat/Projects/satellitepy/exps/tobias_SR/v1/Fainair_Lables_dota_org',
'/home/murat/Projects/satellitepy/exps/tobias_SR/v1/Fainair_Lables_dota_orgx4']

out_folders = ['/home/murat/Projects/satellitepy/exps/tobias_SR/v1_fixed/Fainair_Lables_dota_enhx4',
'/home/murat/Projects/satellitepy/exps/tobias_SR/v1_fixed/Fainair_Lables_dota_enh',
'/home/murat/Projects/satellitepy/exps/tobias_SR/v1_fixed/Fainair_Lables_dota_org',
'/home/murat/Projects/satellitepy/exps/tobias_SR/v1_fixed/Fainair_Lables_dota_orgx4']

In [5]:
for out_folder in out_folders:
    Path(out_folder).mkdir(parents=True, exist_ok=True)

instance_dict = get_satellitepy_table()['coarse-class']
cgc_keys = list(instance_dict.keys())
for in_folder, out_folder in zip(in_folders,out_folders):
    for result_path in Path(in_folder).glob('**/*'):
        with open(result_path, 'r') as f:
            result = json.load(f)
        result['det_labels']["confidence-scores"] = np.array(result['det_labels']["confidence-score"]).astype(float).tolist()
        del result['det_labels']["confidence-score"]
        # result['det_labels']['hbboxes'] = np.array(result['det_labels']['hbboxes']).astype(float).tolist()
        # result['det_labels']['coarse-class'] = [cgc_keys[ind] for ind in np.array(result['det_labels']['coarse-class']).astype(int)]
        result['det_labels']['coarse-class'] = np.array(result['det_labels']['coarse-class']).astype(int).tolist()
        # result['gt_labels']['coarse-class'] = [cgc_keys[ind] for ind in np.array(result['gt_labels']['coarse-class']).astype(int)]
        result['gt_labels']['coarse-class'] = np.array(result['gt_labels']['coarse-class']).astype(int).tolist()
        # result['gt_labels']['hbboxes'] = np.array(result['gt_labels']['hbboxes']).astype(float).tolist()

        out_result_path = Path(out_folder) / result_path.name
        with open(str(out_result_path), 'w') as fw:
            json.dump(result, fw)


        